# Import packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import scipy.stats # For finding skew in the dataset

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Additional package Pycaret for help with model selection
# Commented out once results are taken into account

#!pip install -q pycaret[full]

# Read in Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Unpickle engineered data

df_train = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/ames-housing/Data/train_transformed.pkl')
df_test = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/ames-housing/Data/test_transformed.pkl')
log_target = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/ames-housing/Data/target_transformed.pkl')

# Get test IDs

test_ids = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/ames-housing/Data/testids.pkl')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Scale data

from sklearn.preprocessing import StandardScaler, RobustScaler

scaler = RobustScaler()
scaler.fit(df_train)
df_train1 = pd.DataFrame(scaler.transform(df_train), index=df_train.index, columns=df_train.columns)
df_test1 = pd.DataFrame(scaler.transform(df_test), index=df_test.index, columns=df_test.columns)

# Pycaret to identify top performing algorithms

This code is commented out because it is time comsuming and verbose. 
However, Catboost Regression, Bayesian Ridge, Ridge Regression, and Light Gradient Boosting Machine were identified to be the top performing models. 

In [5]:
#from pycaret.regression import setup, compare_models, models

In [6]:
#_ = setup(data=pd.concat([df_train1, log_target], axis=1), target='SalePrice')

In [7]:
#compare_models()

# ModelContainer class

In [8]:
class ModelContainer():
  def __init__(self):
    self.models = {}
    self.mean_mse = {}
    self.std_mse = {}
    self.best_model = None
    self.kfolds = 0

  def add_model(self, model):
    self.models[model[0]] = model[1]

  def get_results(self, X, y, kfolds=5):
    for model_name, model in self.models.items():
      neg_mse = cross_val_score(model, X, y, cv=kfolds, scoring='neg_mean_squared_error')
      self.mean_mse[model_name] = -1.0*np.mean(neg_mse)
      self.std_mse[model_name] = np.std(neg_mse)
      self.kfolds = kfolds
      print(model_name, 'cross-validated.')

  def select_best_model(self):
    temp = min(self.mean_mse)
    self.best_model = self.models[temp]

  def best_model_fit(self, X_train, y_train):
    self.best_model.fit(X_train, y_train)

  def best_model_predict(self, X_test):
    print('The best individual model is:\n', self.best_model)
    return np.exp(self.best_model.predict(X_test))

  def ensemble_predict(self, X_train, y_train, X_test, weights): 
    if len(self.models) != len(weights):
      print('Number of weights and models should be the same.')
    else:
      predictions = {}
      final_predictions = np.array
      for model_name, model in self.models.items():
        model.fit(X_train, y_train)
        predictions[model_name] = np.exp(model.predict(X_test))

      return sum(weights[model_name] * predictions.get(model_name) for model_name in weights)
      
    



  def print_summary(self):
    print('\nModel Summaries:\n')
    for model_name, model in self.models.items():
      print(model_name, ':')
      print('Mean MSE over', self.kfolds.n_splits, 'folds:',  self.mean_mse[model_name], '+/-', self.std_mse[model_name],'\n')


# Model training

In [9]:
!pip install -q catboost

     |████████████████████████████████| 67.4 MB 29 kB/s 


In [10]:
# Import relevant packages

from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.linear_model import Ridge, BayesianRidge
from mlxtend.regressor import StackingCVRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

### Baseline model

Pycaret indicated that CatBoost Regressor is the best performing model. We will choose this as the baseline and seek to improve on it. 

In [11]:
kf = KFold(n_splits=10)

In [12]:
baseline_model = ModelContainer()
baseline_model.add_model(('Baseline CatBoost', CatBoostRegressor(verbose=0)))

baseline_model.get_results(df_train1, log_target, kfolds=kf)

baseline_model.print_summary()

Baseline CatBoost cross-validated.

Model Summaries:

Baseline CatBoost :
Mean MSE over 10 folds: 0.014940510524764045 +/- 0.004725862229804103 



### Hyperparamter tuning

Tune hyperparameters for the 4 candidate algorithms using RandomizedSearchCV. 

Note that exact parameters will change with each run, but any one set of tuned hyperparameters will get very close to the optimal value so it won't be necessary to modify the hyperparameters everytime.

First, define a function that will accept a model and a corresponding parameter grid and return the best combination using GridSearch or RandomizedSearch.

Then we tune the hyperparameters for each of the algorithms as follows:

* Define the parameter grid for the algorithm with a wide range of parameters

* This first run will get us close to the optimal values. Then modify the parameter grid to narrow down the range and make it more granular

* This should get us close enough to the actual optimal values

In [13]:
def tune_hyperparameters(training_data, training_targets, model, param_grid):

  random_search = RandomizedSearchCV(
      model,
      param_distributions=param_grid,
      scoring='neg_mean_squared_error',
      n_jobs=-1,
      n_iter=20,
      cv=10)

  random_search.fit(np.array(training_data), np.array(training_targets))
  print('Best hyperparameters for', model, 'are:')
  print(random_search.best_params_)
  print('\n')

In [14]:
param_ridge = {'alpha':[150, 175, 200, 225, 250, 275, 300, 325,350]}

param_br = {
    'alpha_1':np.linspace(100, 1000, 100),
    'alpha_2':np.linspace(100, 1000, 100),
    'lambda_1':np.linspace(100,1000, 100),
    'lambda_2':np.linspace(1,100, 100)
}

param_lgbm ={
    'num_leaves': np.arange(1, 100, 5),
    'min_data_in_leaf': np.arange(1,100,1),
    'max_depth': np.arange(1,100,1)
}

param_cbr= {
    'depth': np.arange(1,100,10),
    'learning_rate': [0.01, 0.05, 0.1, 0.5, 1],
    'iterations': np.arange(100,1000,100)
}

In [15]:
tune_hyperparameters(df_train1, log_target, Ridge(), param_ridge)

tune_hyperparameters(df_train1, log_target, BayesianRidge(), param_br)

# Commented out becasue the results are too verbose.
# Tuning was done a few times and the following params were chosen:
# num_leaves: 50, max_depth: 10

#tune_hyperparameters(df_train1, log_target, LGBMRegressor(), param_lgbm)

# Commented out because it is extremely time consuming and verbose
# These will be treated as the final hyperparameters:
# 'iterations': 6000, 'depth': 5, 'learning_rate': 0.005

#tune_hyperparameters(df_train1, log_target, CatBoostRegressor(), param_cbr)

Best hyperparameters for Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001) are:
{'alpha': 150}


Best hyperparameters for BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=True,
              lambda_1=1e-06, lambda_2=1e-06, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False) are:
{'lambda_2': 19.0, 'lambda_1': 527.2727272727273, 'alpha_2': 618.1818181818182, 'alpha_1': 372.72727272727275}




### Retrain tuned models

In [16]:
tuned_models = ModelContainer()

tuned_models.add_model(('Ridge Regression', Ridge(alpha=250)))

tuned_models.add_model(('Bayesian Ridge', BayesianRidge(alpha_1=850, alpha_2=550, lambda_1=500, lambda_2=5)))

tuned_models.add_model(('LGBM', LGBMRegressor(num_leaves=50, max_depth=5)))

tuned_models.add_model(('CatBoost Regressor', CatBoostRegressor(iterations=6000, depth=5, learning_rate=0.005, verbose=0)))

In [17]:
tuned_models.models

{'Bayesian Ridge': BayesianRidge(alpha_1=850, alpha_2=550, alpha_init=None, compute_score=False,
               copy_X=True, fit_intercept=True, lambda_1=500, lambda_2=5,
               lambda_init=None, n_iter=300, normalize=False, tol=0.001,
               verbose=False),
 'CatBoost Regressor': <catboost.core.CatBoostRegressor at 0x7f1dbdbc2dd0>,
 'LGBM': LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=5,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=50, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
 'Ridge Regression': Ridge(alpha=250, copy_X=True, fit_intercept=True, max_iter=None,
       normalize=False, random_state=None, solver='auto', tol=0.001)}

In [18]:
tuned_models.get_results(df_train1, log_target, kfolds=kf)

tuned_models.select_best_model()
tuned_models.best_model_fit(df_train1, log_target)

Ridge Regression cross-validated.
Bayesian Ridge cross-validated.
LGBM cross-validated.
CatBoost Regressor cross-validated.


In [19]:
tuned_models.print_summary()


Model Summaries:

Ridge Regression :
Mean MSE over 10 folds: 0.019510598329142502 +/- 0.006157353678586185 

Bayesian Ridge :
Mean MSE over 10 folds: 0.01651722989747783 +/- 0.005963128503879882 

LGBM :
Mean MSE over 10 folds: 0.01698802403099343 +/- 0.004876987719375172 

CatBoost Regressor :
Mean MSE over 10 folds: 0.014699055506051873 +/- 0.004862934851674393 



### Ensembling

In [20]:
ensemble_weights = {
    'CatBoost Regressor': 0.45,
    'Bayesian Ridge':     0.35,
    'LGBM':               0.15,
    'Ridge Regression':   0.05
}

ensemble_predictions = tuned_models.ensemble_predict(df_train1, log_target, df_test1, ensemble_weights)
best_model_predictions = tuned_models.best_model_predict(df_test1)

The best individual model is:
 BayesianRidge(alpha_1=850, alpha_2=550, alpha_init=None, compute_score=False,
              copy_X=True, fit_intercept=True, lambda_1=500, lambda_2=5,
              lambda_init=None, n_iter=300, normalize=False, tol=0.001,
              verbose=False)


# Submissions

In [21]:
submission_bestmodel = pd.concat([test_ids, pd.Series(best_model_predictions, name='SalePrice')], axis=1)
submission_ensemble = pd.concat([test_ids, pd.Series(ensemble_predictions, name='SalePrice')], axis=1)

In [22]:
# Score with best individual model
# Score on leaderboard: 0.12574 (lower is better)

In [24]:
# Score on leaderboard: 0.12140 (lower is better)
submission_ensemble.to_csv('/content/drive/MyDrive/Colab Notebooks/ames-housing/Data/Submission.csv', index=False)